In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Create dummy model output
batch_size = 2
max_seq_len = 50  # Time steps
num_classes = 5  # (A, T, C, G, blank)

# Random model output logits
logits = torch.randn(batch_size, max_seq_len, num_classes)  # Shape: (batch, seq_len, num_classes)

# Convert logits to log probabilities for CTC Loss
log_probs = F.log_softmax(logits, dim=-1)  # Ensure values sum to 1

# Transpose for CTC Loss (T, N, C)
log_probs = log_probs.permute(1, 0, 2)  # Shape: (seq_len, batch, num_classes)

# Define targets (flattened)
targets = torch.tensor([0, 1, 2, 3, 0, 1, 2, 3])  # Example (A=0, T=1, C=2, G=3)

# Define sequence lengths
input_lengths = torch.tensor(batch_size* [50])  # Both sequences have full length
target_lengths = torch.tensor(batch_size* [4])  # Targets are length 4 each

# Initialize CTC loss function
ctc_loss_fn = nn.CTCLoss(blank=4)  # Assuming blank token is index 4

# Compute CTC Loss
loss = ctc_loss_fn(log_probs, targets, input_lengths, target_lengths)
print("CTC Loss:", loss.item())


CTC Loss: 16.456607818603516
